# stats for mcnair summer 2016 paper

This notebook computes the necessary information and produces plots for the 2016 McNair paper on the neural noise analysis.

In [3]:
%matplotlib inline
import os
import seaborn
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import scipy.stats as stats
import scipy.io
import matplotlib.pyplot as plt
from sklearn import linear_model
mpl.rcParams['figure.figsize'] = (16, 10)

In [4]:
df = pd.read_csv('../data/pipeline-full/ya-oa-full-ransac-2-24-eyesc-eyeso-sep.csv')
oa = df[df.AGE >= 60]
oa = oa[oa.CLASS != 'SA']
ya = df[df.AGE <= 35.5]

In [5]:
oa.describe()

,SUBJECT,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,A1_EYESC,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
count,3.700000e+01,37.000000,37.000000,3.700000e+01,3.700000e+01,3.700000e+01,3.700000e+01,37.000000,37.000000,37.000000,...,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000,37.000000
mean,1.201271e+08,76.227737,1.729730,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.097910,-2.548412,-4.909993,...,-4.436991,-5.490548,-5.699747,-5.414903,-4.357605,-3.788481,-4.158588,-4.564567,-4.267526,-4.829669
std,2.420223e+01,6.154934,0.450225,9.004297e-16,1.800859e-15,9.004297e-16,9.004297e-16,2.307571,3.076342,2.242296,...,2.846882,2.015466,2.304775,2.144134,2.664102,2.829162,2.676524,2.699911,2.364173,2.803253
min,1.201271e+08,64.273790,1.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-8.470149,-10.085858,-8.158394,...,-8.811849,-8.242412,-8.887641,-8.272126,-8.813164,-7.499692,-9.870920,-9.256354,-8.411475,-9.588904
25%,1.201271e+08,72.580000,1.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-4.334330,-4.138652,-6.702094,...,-6.519261,-7.238434,-6.840612,-6.722988,-6.161032,-6.035475,-5.738254,-6.364733,-6.004532,-6.482026
50%,1.201271e+08,74.954140,2.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.300887,-2.482356,-5.487868,...,-5.452952,-5.738814,-6.186407,-6.116207,-5.150202,-4.229259,-4.891438,-5.475763,-4.862521,-5.453839
75%,1.201272e+08,80.626970,2.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-2.029024,-0.451358,-3.097770,...,-1.882340,-4.107982,-4.778081,-4.427154,-2.469806,-1.935509,-1.858008,-2.158916,-2.689703,-2.743728
max,1.201272e+08,87.857630,2.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,2.676874,4.381933,0.928262,...,0.830348,-0.253221,2.215389,0.030454,1.263249,3.074497,1.197417,2.250867,2.446284,1.129387


In [6]:
ya.describe()

/Users/jorge/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,SUBJECT,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,A1_EYESC,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
count,3.100000e+01,31.000000,30.000000,3.100000e+01,3.100000e+01,3.100000e+01,3.100000e+01,31.000000,31.000000,31.000000,...,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,4.376226e+08,29.032258,1.666667,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.811990,-2.917458,-3.869767,...,-4.683189,-5.416235,-5.504162,-5.526392,-4.513075,-4.226709,-4.502851,-4.972432,-4.549598,-5.446568
std,4.794976e+08,3.420369,0.479463,9.028600e-16,1.354290e-15,9.028600e-16,9.028600e-16,2.049548,2.712162,2.601924,...,2.201832,1.840930,1.848924,1.806852,2.380811,3.227592,2.282329,2.626998,2.237804,2.073636
min,1.121181e+08,24.000000,1.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-9.076692,-6.271256,-8.780847,...,-8.670858,-8.117344,-8.789205,-8.074835,-8.859695,-10.213600,-8.840171,-9.344243,-8.096242,-10.018689
25%,1.121185e+08,26.000000,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-5.182282,-5.008430,-5.527335,...,-6.282941,-6.837469,-6.731886,-6.826478,-5.783237,-6.592646,-6.370141,-6.818289,-5.920706,-7.223858
50%,1.121186e+08,29.000000,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.875112,-3.172496,-3.801026,...,-4.678928,-5.982098,-5.873277,-5.980982,-4.467714,-4.562526,-4.570710,-5.224611,-4.819081,-5.304434
75%,1.121181e+09,31.000000,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-2.192592,-1.729513,-2.294716,...,-3.373199,-4.164744,-4.472345,-4.288519,-3.024341,-2.427540,-3.445535,-3.557182,-2.827172,-4.270882
max,1.121182e+09,35.000000,2.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,0.352321,3.451306,2.384513,...,-0.125437,-1.553612,-1.683475,-1.699259,1.054327,3.096599,1.222900,2.042032,1.091900,-1.188407


In [7]:
df.describe()

/Users/jorge/anaconda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,SUBJECT,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,A1_EYESC,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
count,9.000000e+01,90.000000,89.000000,9.000000e+01,9.000000e+01,9.000000e+01,9.000000e+01,90.000000,90.000000,90.000000,...,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,2.626772e+08,59.696629,1.674157,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.313822,-2.384447,-4.328958,...,-4.170733,-5.035832,-5.311578,-5.075529,-4.088287,-3.665790,-3.987798,-4.398891,-4.049660,-4.674085
std,3.547440e+08,24.431205,0.471344,8.931542e-16,3.126040e-15,1.071785e-14,7.145234e-15,2.204149,2.905146,2.432849,...,2.738751,2.149917,2.089702,2.106723,2.613658,2.881469,2.543300,2.936167,2.381630,2.571158
min,1.121181e+08,24.000000,1.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-9.076692,-10.085858,-9.030100,...,-8.811849,-8.242412,-8.887641,-8.272126,-8.859695,-10.213600,-9.870920,-9.344243,-8.411475,-10.018689
25%,1.141209e+08,31.000000,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-4.643339,-4.383682,-6.289111,...,-6.315433,-6.721175,-6.767204,-6.668633,-5.947895,-6.095441,-5.677457,-6.597369,-5.765156,-6.043802
50%,1.201271e+08,72.440000,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-3.458205,-2.584938,-4.413796,...,-4.674613,-5.444685,-5.837880,-5.527094,-4.457713,-3.826100,-4.399621,-4.989265,-4.549384,-5.130916
75%,1.201272e+08,80.826828,NaN,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,-2.051051,-0.362097,-2.789614,...,-1.925368,-3.888528,-4.040153,-3.809654,-2.570162,-1.457035,-2.119171,-2.142276,-2.628656,-3.009987
max,1.121182e+09,91.578370,2.000000,-3.212677e+00,-2.264243e+00,-5.096711e+00,-4.183642e+00,3.093442,4.381933,2.532097,...,2.042491,0.791619,2.215389,1.085539,2.471565,3.096599,1.862275,2.933310,2.446284,1.840816


In [8]:
df[df.CLASS.isin(['MCI'])].describe()

,SUBJECT,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,A1_EYESC,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
count,9.000000e+00,9.000000,9.000000,9.000000e+00,9.000000,9.000000e+00,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,1.201271e+08,72.104497,1.777778,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,-3.248603,-3.596824,-5.055044,...,-5.466124,-5.698331,-5.810255,-5.821667,-5.011123,-4.403882,-4.905399,-5.170162,-4.984992,-5.494744
std,1.330205e+01,5.297370,0.440959,4.710277e-16,0.000000,9.420555e-16,0.000000,2.192533,3.343498,2.379238,...,3.007608,2.400423,2.371387,2.276981,2.755089,2.806837,1.836710,2.335235,1.957643,2.944085
min,1.201271e+08,64.273790,1.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,-7.384566,-8.461916,-7.831300,...,-8.811849,-8.242412,-8.452708,-7.823318,-8.163080,-6.724240,-7.211276,-8.570212,-7.749446,-8.439735
25%,1.201271e+08,70.507870,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,-4.256647,-4.361589,-6.757729,...,-6.533744,-7.490177,-7.085629,-6.730450,-6.359039,-6.529305,-5.959385,-6.364733,-6.299772,-7.947096
50%,1.201271e+08,71.734430,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,-3.172178,-3.916043,-5.695389,...,-6.429221,-6.015081,-6.613410,-6.557847,-5.933240,-5.977841,-5.701384,-5.475763,-5.212359,-6.031898
75%,1.201271e+08,74.954140,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,-2.188560,-2.482356,-3.509178,...,-3.542725,-5.043177,-3.968736,-5.660722,-4.933817,-1.943243,-3.931538,-3.388651,-3.679989,-4.296672
max,1.201272e+08,80.626970,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642,0.690187,3.302449,-0.606450,...,0.830348,-0.253221,-1.056013,-0.370737,1.263249,1.105808,-1.130404,-1.639727,-1.272261,1.129387


In [9]:
df[df.CLASS.isin(['SA'])].describe()

,SUBJECT,AGE,SEX,AVG_OA_PSD_EYESC,AVG_OA_PSD_EYESO,AVG_YA_PSD_EYESC,AVG_YA_PSD_EYESO,AVG_PSD_EYESC,AVG_PSD_EYESO,A1_EYESC,...,B26_EYESO,B27_EYESO,B28_EYESO,B29_EYESO,B30_EYESO,FRONTAL_EYESO,LTEMPORAL_EYESO,CENTRAL_EYESO,RTEMPORAL_EYESO,OCCIPITAL_EYESO
count,1.700000e+01,17.000000,17.000000,1.700000e+01,17.000000,1.700000e+01,1.700000e+01,17.000000,17.000000,17.000000,...,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000
mean,1.201271e+08,84.545314,1.588235,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,-3.064900,-0.848630,-3.733577,...,-2.578061,-3.615126,-4.431768,-3.777716,-2.708744,-2.023188,-2.633167,-3.298698,-2.271684,-3.031488
std,1.568790e+01,2.834529,0.507300,9.155134e-16,0.000000,9.155134e-16,9.155134e-16,2.413310,2.638951,2.635524,...,3.234576,2.501456,1.846157,1.935669,2.681571,2.014222,2.355096,3.029259,2.142904,2.346553
min,1.201271e+08,80.465430,1.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,-6.895460,-4.750274,-9.030100,...,-8.165078,-7.709847,-7.574430,-6.771223,-6.506009,-6.311474,-6.539585,-8.832658,-5.937649,-6.662115
25%,1.201271e+08,83.340180,1.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,-4.512241,-2.447626,-5.236688,...,-5.235552,-5.366292,-5.811117,-4.929753,-5.022007,-2.579495,-4.410924,-4.946171,-3.587571,-4.947936
50%,1.201271e+08,84.084870,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,-3.100784,-1.665514,-3.684359,...,-2.889535,-3.880798,-4.366260,-3.842434,-3.110215,-1.575350,-2.449469,-4.058723,-2.637770,-3.190228
75%,1.201271e+08,84.974670,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,-2.669409,1.243859,-2.669808,...,0.255030,-2.145809,-2.669934,-2.771980,-0.887550,-1.225921,-1.275522,-0.545920,-0.181946,-2.056348
max,1.201272e+08,91.578370,2.000000,-3.212677e+00,-2.264243,-5.096711e+00,-4.183642e+00,3.093442,4.045713,2.532097,...,2.042491,0.791619,-1.917072,1.085539,2.471565,1.255704,1.862275,2.933310,1.324665,1.840816


In [10]:
channels = ["A1","A2","A3","A4","A5","A6","A7","A8","A10","A11","A12","A13","A14","A15","A16","A17","A18","A21","A22","A23","A24","A25","A26","A27","A29","A30","A31","B1","B2","B3","B4","B5","B6","B8","B9","B10","B11","B12","B13","B14","B17","B18","B19","B20","B21","B22","B23","B24","B26","B27","B28","B29","B30"]

In [24]:
male = 0
female = 0
for j in list(ya.SEX):
    if j == 1:
        male += 1
    elif j == 2:
        female += 1

In [26]:
male

10

In [27]:
female

20

In [30]:
len(list(ya.SEX))

31

In [31]:
male = 0
female = 0
for j in list(oa.SEX):
    if j == 1:
        male += 1
    elif j == 2:
        female += 1

In [32]:
male

10

In [33]:
female

27

In [34]:
len(list(oa.SEX))

37